In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/university-students-complaints-and-reports/Datasetprojpowerbi.csv


In [2]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
from nltk.tokenize import word_tokenize
nltk.download("punkt")
from nltk.stem import PorterStemmer
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
import gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel

df = pd.read_csv("/kaggle/input/university-students-complaints-and-reports/Datasetprojpowerbi.csv")

df["Reports_lower"] = df["Reports"].str.lower()

translator = str.maketrans(" "," ",string.punctuation)
df["Reports_nopunct"] = df["Reports_lower"].str.translate(translator)

english_stopwords = set(stopwords.words("english"))
custom_stopwords = {"im","really","student","like","feel","ive"}
combined_stopwords = english_stopwords.union(custom_stopwords)
def remove_stopwords(text):
    words = word_tokenize(text.lower())
    improved_text = [word for word in words if word not in combined_stopwords and word not in translator and not word.isdigit()]
    return " ".join(improved_text)

df["cleaned_Reports"] = df["Reports_nopunct"].apply(remove_stopwords)

stemmer = PorterStemmer()
def stem_text(text):
    words = word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in words]
    return " ".join(stemmed_words)

df["stemmed_Reports"] = df["cleaned_Reports"].apply(stem_text)

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["stemmed_Reports"])
tfidf_df = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names_out())

sum_tfidf = np.array(tfidf_df.sum(axis=0)).flatten()
words_tfidf_df = pd.DataFrame(list(zip(tfidf_df,sum_tfidf)), columns=["word", "tfidf_score"])
most_frequent_words = words_tfidf_df.sort_values(by="tfidf_score", ascending=False)
print(" ")
print("Most Frequent Words incl. TfIdf Score:")
print(most_frequent_words.head(10))
print(" ")

n_topics = 11
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda.fit(X)
feature_names = vectorizer.get_feature_names_out()

for topic_index, topic in enumerate(lda.components_):
    print("Extracted Topics:")
    print(f"Topic #{topic_index}:")
    print(" ".join([feature_names[i] for i in topic.argsort()[-10:]]))

doc_topic_dist = lda.transform(X)
topic_prevalence = doc_topic_dist.sum(axis=0)
topic_df = pd.DataFrame({"Topic": range(lda.n_components),"Prevalence": topic_prevalence})

topic_df = topic_df.sort_values(by="Prevalence", ascending=False)

print(" ")
print("Extracted Topics with Prevalence:")
print(doc_topic_dist[0])
print(topic_df)
print(" ")

tokenized_docs = [word_tokenize(text) for text in df["stemmed_Reports"]]
dictionary = Dictionary(tokenized_docs)
corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]

n_topics = 11
lda_gensim = LdaModel(corpus, num_topics=n_topics, id2word=dictionary, random_state=42)

for topic_index, topic in lda_gensim.print_topics(num_words=5):
    print("Topics with Coherence Scores:")
    print(f"Topic #{topic_index}: {topic}")

coherence_model_lda = CoherenceModel(model=lda_gensim, texts=tokenized_docs, dictionary=dictionary, coherence="c_v")
coherence_score = coherence_model_lda.get_coherence()
print(" ")
print(f"Coherence Score: {coherence_score}")



[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
 
Most Frequent Words incl. TfIdf Score:
         word  tfidf_score
1103  student    36.850763
1228  univers    28.569606
8      access    28.494155
5      academ    28.289367
655     limit    27.049631
439      find    26.602705
786    option    26.039072
83      avail    25.734455
688      make    25.623847
744      need    25.107747
 
Extracted Topics:
Topic #0:
databas materi pay troubl research frustrat caus job access lot
Extracted Topics:
Topic #1:
motiv time student offer avail cafeteria stay campu food option
Extracted Topics:
Topic #2:
network profession career advisor student univers languag job academ opportun
Extracted Topics:
Topic #3:
ix student program concern import cultur appreci univers team sport
Extracted Topics:
Topic #4:
spend outrag n